In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import json
from openai import OpenAI
import csv
import re
from random import randint
import warnings
import openai
import yaml
warnings.filterwarnings('ignore')

In [24]:
def load_config(yaml_path="P4-config.yaml"):
    with open(yaml_path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)
    
config = load_config()


In [3]:
load_dotenv()

client_gpt = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [4]:
# load chatgpt prompts
chatgpt_prompts_dir = config["prompts_dir"]["chatgpt_prompt_dir"]
basic_prompt_path = os.path.join(chatgpt_prompts_dir, "basic_realworld_test")
onco_prompt_path = os.path.join(chatgpt_prompts_dir, "onco_realworld_test")
reviewer_prompt_path = os.path.join(chatgpt_prompts_dir, "reviewer_realworld_test")

basic_filepath = basic_prompt_path +  '/pdsqi_input_to_llm_as_a_judge_zero_shot.csv'
onco_filepath = onco_prompt_path +  '/pdsqi_input_to_llm_as_a_judge_zero_shot.csv'
reviewer_filepath = reviewer_prompt_path +  '/pdsqi_input_to_llm_as_a_judge_zero_shot.csv'

df_basic_prompts = pd.read_csv(basic_filepath)
df_onco_prompts = pd.read_csv(onco_filepath)
df_reviewer_prompts = pd.read_csv(reviewer_filepath)

    


In [15]:
df_basic_prompts.head()

,record_id,prompt
0,0,Here is your new role and persona:\n Yo...
1,1,Here is your new role and persona:\n Yo...
2,2,Here is your new role and persona:\n Yo...
3,3,Here is your new role and persona:\n Yo...
4,4,Here is your new role and persona:\n Yo...


In [5]:
df_basic = pd.read_csv(config["input_file_paths"]["basic_realworld_test"])
df_onco = pd.read_csv(config["input_file_paths"]["onco_realworld_test"])
df_reviewer = pd.read_csv(config["input_file_paths"]["reviewer_realworld_test"])

# add index columns
def add_index_column(df):
    df.reset_index(inplace=True)
    df.rename(columns={"index": "record_id"}, inplace=True)
    return df

df_basic = add_index_column(df_basic)
df_onco = add_index_column(df_onco)
df_reviewer = add_index_column(df_reviewer)

In [6]:
df_basic.head()

,record_id,Drug,Receptor,PDB_ID,generated_report,prompt_tier
0,0,Metformin,Acetyl-CoA carboxylase 2,"3FF6,3TDC,2X24,3JRX,3JRW,2HJW,4HQ6,5KKN,3GLK,3...","{\n ""Mechanism"": ""Metformin is an oral anti-h...",basic
1,1,Pioglitazone,Peroxisome proliferator-activated receptor gamma,"3E00,3DZY,3DZU,7QB1,6L89,6K0T,6AD9,5HZC,5F9B,5...","<Output>\n{\n ""Mechanism"": ""Pioglitazone is a...",basic
2,2,Alogliptin,Dipeptidyl peptidase 4,"2QTB,2QT9,2BGR,2JID,3F8S,2QJR,3W2T,3VJM,3VJL,3...","<Output>\n{\n ""Mechanism"": ""Alogliptin is a D...",basic
3,3,Linagliptin,Dipeptidyl peptidase 4,"2QTB,2QT9,2BGR,2JID,3F8S,2QJR,3W2T,3VJM,3VJL,3...","<Output>\n{\n ""Mechanism"": ""Linagliptin is a ...",basic
4,4,Sitagliptin,Dipeptidyl peptidase 4,"2QTB,2QT9,2BGR,2JID,3F8S,2QJR,3W2T,3VJM,3VJL,3...","{\n ""Mechanism"": ""Sitagliptin is a DPP-4 inhi...",basic


In [7]:
def create_scorefile(df, name):
    scoreFile_name = f"scoreFile_{name}"
    scoreFile_name = pd.DataFrame({
        "record_id": df["record_id"],
        "Drug": df["Drug"],
        "Summary": df["generated_report"],
        "accurate": None,
        "organized": None,
        "comprehensible": None,
        "succinct": None,
    })
    scoreFile_name.head()

    return scoreFile_name

basic_scorefile = create_scorefile(df_basic, "basic")
onco_scorefile = create_scorefile(df_onco, "onco")
reviewer_scorefile = create_scorefile(df_reviewer, "reviewer")

basic_scorefile.head()
    

,record_id,Drug,Summary,accurate,organized,comprehensible,succinct
0,0,Metformin,"{\n ""Mechanism"": ""Metformin is an oral anti-h...",None,None,None,None
1,1,Pioglitazone,"<Output>\n{\n ""Mechanism"": ""Pioglitazone is a...",None,None,None,None
2,2,Alogliptin,"<Output>\n{\n ""Mechanism"": ""Alogliptin is a D...",None,None,None,None
3,3,Linagliptin,"<Output>\n{\n ""Mechanism"": ""Linagliptin is a ...",None,None,None,None
4,4,Sitagliptin,"{\n ""Mechanism"": ""Sitagliptin is a DPP-4 inhi...",None,None,None,None


In [8]:
def call_o4mini_chat(prompt: str) -> str:
    """
    Calls o4-mini via chat.completions and returns the text content.
    Your prompt already includes all instructions and ends with OUTPUT:.
    """
    if prompt is None or (isinstance(prompt, float) and pd.isna(prompt)):
        raise ValueError("Prompt is empty or NaN")

    resp = client_gpt.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=6000,
    )

    # Standard ChatCompletion access pattern
    return resp.choices[0].message.content

In [21]:
def extract_rating(df, scoreFile):
    raw_outputs = []
    errors = []

    # Make updates easier/faster by indexing scoreFile on record_id
    if "record_id" not in scoreFile.columns:
        raise ValueError("scoreFile is missing 'record_id' column")
    
    scoreFile_indexed = scoreFile.set_index("record_id")

    for i, row in tqdm(df.iterrows(), total=len(df)):
        record_id = row["record_id"]
        prompt = row["prompt"]

        # handle empty / NaN prompts
        if prompt is None or (isinstance(prompt, float) and pd.isna(prompt)):
            errors.append({
                "index": i,
                "record_id": record_id,
                "error": "Empty or NaN prompt"
            })
            continue  # IMPORTANT: skip calling the model for this row

        try:
            raw = call_o4mini_chat(prompt)
            raw_outputs.append({"record_id": record_id, "raw_output": raw})

            # Try to parse JSON
            try:
                scores = json.loads(raw)
            except Exception as je:
                errors.append({
                    "index": i,
                    "record_id": record_id,
                    "error": f"JSON parse failed: {je}",
                    "raw_output": raw,
                })
                continue

            # Update scoreFile_indexed with parsed scores
            for col in ["accurate", "organized", "comprehensible", "succinct"]:
                if col in scores:
                    scoreFile_indexed.loc[record_id, col] = scores[col]

        except Exception as e:
            errors.append({
                "index": i,
                "record_id": record_id,
                "error": f"API call failed: {e}",
            })

    # --- turn everything into DataFrames and RETURN them ---
    updated_scores = scoreFile_indexed.reset_index()
    raw_df = pd.DataFrame(raw_outputs)
    errors_df = pd.DataFrame(errors)

    return updated_scores, raw_df, errors_df


In [22]:
basic_scores, basic_raw, basic_errors = extract_rating(
    df_basic_prompts,
    basic_scorefile
)

onco_scores, onco_raw, onco_errors = extract_rating(
    df_onco_prompts,
    onco_scorefile

)

reviewer_scores, reviewer_raw, reviewer_errors = extract_rating(
    df_reviewer_prompts,
    reviewer_scorefile
)


100%|██████████| 29/29 [03:08<00:00,  6.49s/it]


In [26]:
output_path = config["output_file_paths"]["chatgpt_ratings_output_dir"]
os.makedirs(output_path, exist_ok=True)

# create output directories for each evaluation set
basic_output_path = os.path.join(output_path, "basic_realworld_test")
os.makedirs(basic_output_path, exist_ok=True)
onco_output_path = os.path.join(output_path, "onco_realworld_test")
os.makedirs(onco_output_path, exist_ok=True)
reviewer_output_path = os.path.join(output_path, "reviewer_realworld_test")
os.makedirs(reviewer_output_path, exist_ok=True)

# Save basic results
basic_scores.to_csv(os.path.join(basic_output_path, "chatgpt_ratings_basic_realworld_test.csv"), index=False)
basic_raw.to_csv(os.path.join(basic_output_path, "chatgpt_raw_outputs_basic_realworld_test.csv"), index=False)
basic_errors.to_csv(os.path.join(basic_output_path, "chatgpt_errors_basic_realworld_test.csv"), index=False)

# Save onco results
onco_scores.to_csv(os.path.join(onco_output_path, "chatgpt_ratings_onco_realworld_test.csv"), index=False)
onco_raw.to_csv(os.path.join(onco_output_path, "chatgpt_raw_outputs_onco_realworld_test.csv"), index=False)
onco_errors.to_csv(os.path.join(onco_output_path, "chatgpt_errors_onco_realworld_test.csv"), index=False)

# Save reviewer results
reviewer_scores.to_csv(os.path.join(reviewer_output_path, "chatgpt_ratings_reviewer_realworld_test.csv"), index=False)
reviewer_raw.to_csv(os.path.join(reviewer_output_path, "chatgpt_raw_outputs_reviewer_realworld_test.csv"), index=False)
reviewer_errors.to_csv(os.path.join(reviewer_output_path, "chatgpt_errors_reviewer_realworld_test.csv"), index=False)